In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read in Data


* Must have the 'Ridership Model Data.csv' in the project folder;
* Create model_data for training the model which has non-relevant features dropped; and
* Create features to be used for generating future feature values.



In [ ]:
# Read in data and split in to training and prediction sets

df = pd.read_csv('Ridership Model Data.csv')
model_data = df.drop(['ISOWeek','Month','Start of Week','Population Growth Rate',
                      'critical cases','BC Vaccination Rate'], axis=1)
features = model_data.drop(['Total Boardings'], axis=1)

# Function to get Test Data

* Choose a year to generate values for;
* Select whether the data should reflect pre- or post-covid scenarios; and
* Select whether the data should include 20,000 hours of expansion.

## Methods

* Data is generally generated using a normal distribution with a mean and standard deviation taken from the true data set;
* Some exceptions are the:
  * Year which is the specified year; and
  * Restaurant Bookings which follows a logarithmic distribution.



In [ ]:
rb_coefs = np.polyfit(np.log(features[features['hospitalizations'] > 0].index.values),
                      features[features['hospitalizations'] > 0]['Restaurant Bookings'],1)

def get_test_data(year,pre_covid=False,expansion=True):
  '''
  Generate randomized data for a specific year.
  '''
  if year > 2021:
    inc = year - 2021
  else:
    inc = 0

  if pre_covid:
    year = [2019] * 52
  else:
    year = [2021] * 52
  if expansion:
    revenue_hours = np.random.normal(features.describe()['Revenue Hours']['mean'] + inc * 20000 / 52,
                                     features.describe()['Revenue Hours']['std'], size=52)
  else:
    revenue_hours = np.random.normal(features.describe()['Revenue Hours']['mean'],
                                     features.describe()['Revenue Hours']['std'], size=52)
  if pre_covid:
    restaurant_bookings = np.random.normal(0,
                                           features[features['hospitalizations'] > 0].describe()['Restaurant Bookings']['std'],
                                           size=52)
  else:
    restaurant_bookings = np.random.normal(rb_coefs[0] * np.log(range(104 + inc * 52,156 + inc * 52)) + rb_coefs[1],
                                           features.describe()['Restaurant Bookings']['std'])
  gas_price = np.random.normal(features.describe()['Gas Price (C/L)']['mean'],
                               features.describe()['Gas Price (C/L)']['std'],size=52)
  if np.random.randn() > 0 or pre_covid:
    uni_season = features[features['ISOYear'] == 2019]['University School Season']
  else:
    uni_season = features[features['ISOYear'] == 2021]['University School Season']
  employment = np.random.normal(features.describe()['Employment']['mean'] * (1 + inc * .01),
                                features.describe()['Employment']['std'],size=52)
  if pre_covid:
    wfh = np.random.normal(features[features['ISOYear'] == 2019].describe()['WFH']['mean'],
                           features[features['ISOYear'] == 2019].describe()['WFH']['std'],size=52)
  else:
    wfh = np.random.normal(features.describe()['WFH']['mean'],features.describe()['WFH']['std'],size=52)
  if pre_covid:
    hospitalizations = np.zeros(52)
  else:
    hospitalizations = np.random.normal(features[features['hospitalizations'] > 0].describe()['hospitalizations']['mean'],
                                        features[features['hospitalizations'] > 0].describe()['hospitalizations']['std'],
                                        size=52)

  df = np.array([year, revenue_hours, restaurant_bookings, gas_price, uni_season, employment, wfh, hospitalizations]).transpose()
  df = pd.DataFrame(data=df,columns=features.columns)

  return df

# Train Test Split

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Scale the data for model training.
X = model_data.drop('Total Boardings',axis=1)
y = model_data['Total Boardings']

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Create and Train the Model

* Hyperparameters have been selected based on previous testing of the model.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Deep fully-connected neural network consisting of four layers (8 -> 5 -> 3 -> 1).
# Rectified linear unit activation functions for non-ouput layers.
# No dropouts as this hindered performance in the model testing.
# Adam optimizer and mse loss function for regression on a single output variable.
model = Sequential()

model.add(Dense(8,activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(3,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='Adam', loss='mse')

In [ ]:
# Fit the model to all our data - 200 epochs chosen based on test runs with validation data.
model.fit(x=X,y=y,batch_size=1,epochs=200)

Epoch 1/200
157/157 [==============================] - 1s 2ms/step - loss: 210358943744.0000
Epoch 2/200
157/157 [==============================] - 0s 2ms/step - loss: 210355666944.0000
Epoch 3/200
157/157 [==============================] - 0s 2ms/step - loss: 210333696000.0000
Epoch 4/200
157/157 [==============================] - 0s 2ms/step - loss: 210240503808.0000
Epoch 5/200
157/157 [==============================] - 0s 1ms/step - loss: 209957470208.0000
Epoch 6/200
157/157 [==============================] - 0s 2ms/step - loss: 209354997760.0000
Epoch 7/200
157/157 [==============================] - 0s 2ms/step - loss: 208284254208.0000
Epoch 8/200
157/157 [==============================] - 0s 2ms/step - loss: 206613037056.0000
Epoch 9/200
157/157 [==============================] - 0s 2ms/step - loss: 204191105024.0000
Epoch 10/200
157/157 [==============================] - 0s 2ms/step - loss: 200916238336.0000
Epoch 11/200
157/157 [==============================] - 0s 2ms/step -

# Function to Generate Predictions

* Generate 100 predictions per week with a given scenario.

In [ ]:
def get_preds(year,pre_covid=False,expansion=True):
  '''
  Generate 100 predictions with the given features for a specified year.
  '''
  predictions = pd.DataFrame(columns=['Week', 'Prediction'])

  if year > 2021:
    inc = year - 2022
  else:
    inc = 0
  
  for i in range(0,100):
    test_data = get_test_data(year,pre_covid,expansion)
    test_data = scaler.transform(test_data)
    test_df = pd.DataFrame()
    test_df['Week'] = range(0 + inc * 52,52 + inc * 52)
    test_df['Prediction'] = model.predict(test_data)
    predictions = pd.concat([predictions, test_df],ignore_index=True)


  return predictions

In [ ]:
pcwe_2022 = get_preds(2022)
pcwe_2023 = get_preds(2023)
pcwe_2024 = get_preds(2024)
pcne_2022 = get_preds(2022,expansion=False)
pcne_2023 = get_preds(2023,expansion=False)
pcne_2024 = get_preds(2024,expansion=False)
ncwe_2022 = get_preds(2022,pre_covid=True)
ncwe_2023 = get_preds(2023,pre_covid=True)
ncwe_2024 = get_preds(2024,pre_covid=True)
ncne_2022 = get_preds(2022,pre_covid=True,expansion=False)
ncne_2023 = get_preds(2023,pre_covid=True,expansion=False)
ncne_2024 = get_preds(2024,pre_covid=True,expansion=False)

pcwe = pd.concat([pcwe_2022,pcwe_2023,pcwe_2024],ignore_index=True)
pcne = pd.concat([pcne_2022,pcne_2023,pcne_2024],ignore_index=True)
ncwe = pd.concat([ncwe_2022,ncwe_2023,ncwe_2024],ignore_index=True)
ncne = pd.concat([ncne_2022,ncne_2023,ncne_2024],ignore_index=True)

In [ ]:
predictions = pd.concat([pcwe,pcne['Prediction'],ncwe['Prediction'],ncne['Prediction']],axis=1,ignore_index=True)
predictions.columns = ['Week','Post-Covid w/ Expansion','Post-Covid no Expansion',
                       'Pre-Covid w/ Expansion','Pre-Covid no Expansion']

In [ ]:
predictions['Week'] = predictions['Week'].apply(lambda x: x + 156)

In [ ]:
true_2022 = pd.DataFrame([248779, 384142, 396384, 439561, 452715, 461370, 461929, 356563])
true_data = pd.concat([model_data['Total Boardings'], true_2022],ignore_index=True)
true_data.columns = ['Total Boardings']

# Plot Results

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
fig = px.scatter(predictions,
                 template='none',
                 x='Week',
                 y=['Week','Post-Covid w/ Expansion','Post-Covid no Expansion','Pre-Covid w/ Expansion',
                    'Pre-Covid no Expansion'],
                trendline='lowess',trendline_options=dict(frac=.2),opacity=0,title='Total Boardings Predictions',
                labels={
                          'Week' : 'Year',
                          'value' : 'Predicted Weekly Boardings',
                          'variable' : 'Scenario'
                      })
fig.update_xaxes(
    ticktext=['2019', '2020', '2021', '2022', '2023', '2024'],
    tickvals=[51, 103, 155, 207, 259, 312],
  )
fig.data = [t for t in fig.data if t.mode == 'lines']
fig.update_traces(showlegend=True, selector=dict(mode='lines'))
fig.add_traces(go.Scatter(name='True Data',x=true_data.index.values,y=true_data['Total Boardings']))
fig.update_traces(hovertemplate='Predicted Boardings: %{y}')
fig.add_traces(go.Scatter(name='Post-Covid w/ Expansion',showlegend=False,line=dict(color='lightblue',width=0),
                          x=np.concatenate([pcwe.groupby('Week').describe()['Prediction'].index.values,
                                            pcwe.groupby('Week').describe()['Prediction'].index.values[::-1]]) + 156,
                          y=pd.concat([pcwe.groupby('Week').describe()['Prediction']['75%'],
                                       pcwe.groupby('Week').describe()['Prediction']['25%'][::-1]]), fill='toself'))
fig.add_traces(go.Scatter(name='Post-Covid no Expansion',showlegend=False,line=dict(color='salmon',width=0),
                          x=np.concatenate([pcne.groupby('Week').describe()['Prediction'].index.values,
                                            pcne.groupby('Week').describe()['Prediction'].index.values[::-1]]) + 156,
                          y=pd.concat([pcne.groupby('Week').describe()['Prediction']['75%'],
                                       pcne.groupby('Week').describe()['Prediction']['25%'][::-1]]), fill='toself'))
fig.add_traces(go.Scatter(name='Pre-Covid w/ Expansion',showlegend=False,line=dict(color='lightgreen',width=0),
                          x=np.concatenate([ncwe.groupby('Week').describe()['Prediction'].index.values,
                                            ncwe.groupby('Week').describe()['Prediction'].index.values[::-1]]) + 156,
                          y=pd.concat([ncwe.groupby('Week').describe()['Prediction']['75%'],
                                       ncwe.groupby('Week').describe()['Prediction']['25%'][::-1]]), fill='toself'))
fig.add_traces(go.Scatter(name='Pre-Covid no Expansion',showlegend=False,line=dict(color='mediumvioletred',width=0),
                          x=np.concatenate([ncne.groupby('Week').describe()['Prediction'].index.values,
                                            ncne.groupby('Week').describe()['Prediction'].index.values[::-1]]) + 156,
                          y=pd.concat([ncne.groupby('Week').describe()['Prediction']['75%'],
                                       ncne.groupby('Week').describe()['Prediction']['25%'][::-1]]), fill='toself'))
fig.data = fig.data[::-1]
fig.show()